# Shuttle Beam (Proof Mass)

Central moving proof mass with 2D release hole arrays.

Adapted from Sirui's reference design (`draw_proofmass`, `draw_proofmass_side`).

In [ ]:
import sys
sys.path.insert(0, "../../libraries")

from mcw_custom_optical_mems_pdk import PDK
PDK.activate()

from shuttle_beam import make_proofmass, make_proofmass_side

## Centered Proof Mass

In [ ]:
pm = make_proofmass(20, 100, 5, 0.8)
pm.plot()

## Side-Aligned Proof Mass (for comb drive holders)

In [ ]:
ps = make_proofmass_side(40, 3, 2, 0.8)
ps.plot()